In [11]:
import heapq

In [12]:
def misplaced_tiles(state):
    return sum([1 for i in range(TILES_NUM) if state[i] != 0 and state[i] != GOAL[i]])

def manhattan_distance(state):
    distance = 0
    for i, tile in enumerate(state):
        if tile != 0:
            goal_tile = GOAL.index(tile)
            distance += abs(i // ROW_COL_LENGTH - goal_tile // ROW_COL_LENGTH) + abs(i % ROW_COL_LENGTH - goal_tile % ROW_COL_LENGTH)
    return distance

In [13]:
# Plansza 3x3
GOAL = (1, 2, 3, 4, 5, 6, 7, 8, 0)
# Plansza 4x4
#GOAL = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
# Powrót z planszy 4x4 do planszy 3x3
#GOAL = (9,1,2,3,10,4,5,6,11,7,8,0,12,13,14,15)

In [14]:
TILES_NUM = len(GOAL)
ROW_COL_LENGTH = int(TILES_NUM**0.5)

def possible_moves(state):
    neighbors = []
    zero_index = state.index(0)
    zero_row = zero_index // ROW_COL_LENGTH
    zero_col = zero_index % ROW_COL_LENGTH

    moves = [(-1, 0, "Góra"), (1, 0, "Dół"), (0, -1, "Lewo"), (0, 1, "Prawo")]
    for row_move, col_move, name in moves:
        row = zero_row + row_move
        col = zero_col + col_move
        if 0 <= row < ROW_COL_LENGTH and 0 <= col < ROW_COL_LENGTH:
            new_index = ROW_COL_LENGTH * row + col
            new_state = list(state)
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            neighbors.append((tuple(new_state), name))
    return neighbors

def is_solvable(state):
    flat = [tile for tile in state if tile != 0]
    inv = 0
    for i in range(len(flat)):
        for j in range(i+1, len(flat)):
            if flat[i] > flat[j]:
                inv += 1
    return inv % 2 == 0

In [15]:
def a_star(start, heuristic):
    # Kolejka: (funkcja stanu, koszt, stan)
    heap = []
    heapq.heappush(heap, (heuristic(start), 0, start))

    g_scores = {start: 0}
    came_from = {}
    nodes_visited = 0

    while heap:
        f, g, current = heapq.heappop(heap)
        nodes_visited += 1

        # Znalazł rozwiązanie
        if current == GOAL:
            path = []
            node = current
            while node in came_from:
                parent, move = came_from[node]
                path.append(move)
                node = parent
            path.reverse()
            return path, nodes_visited, None
        
        # Przeszukwanie sąsiadów
        for neighbor, move in possible_moves(current):
            new_g = g + 1
            if neighbor not in g_scores or new_g < g_scores[neighbor]:
                g_scores[neighbor] = new_g
                came_from[neighbor] = (current, move)
                f_neighbor = new_g + heuristic(neighbor)
                heapq.heappush(heap, (f_neighbor, new_g, neighbor))

        # Nie znalazł rozwiązania
    return None, nodes_visited, "Nie znaleziono rozwiązania."

In [16]:
state = (0,1,2,3,4,5,6,7,8)

moves, num_nodes, error = a_star(state, manhattan_distance)
if error:
    print("Błąd:", error)
else:
    print("Liczba odwiedzonych węzłów:", num_nodes)
    print("Długość rozwiązania:", len(moves))
    print("Sekwencja ruchów od obecnego stanu do rozwiązania:")
    for i, move in enumerate(moves):
        print(f"{i+1}. {move}")

Liczba odwiedzonych węzłów: 1349
Długość rozwiązania: 22
Sekwencja ruchów od obecnego stanu do rozwiązania:
1. Prawo
2. Dół
3. Lewo
4. Dół
5. Prawo
6. Prawo
7. Góra
8. Lewo
9. Lewo
10. Dół
11. Prawo
12. Góra
13. Góra
14. Prawo
15. Dół
16. Dół
17. Lewo
18. Lewo
19. Góra
20. Prawo
21. Prawo
22. Dół


In [ ]:
import itertools

max_length = -1
hardest_state = None
hardest_moves = None
hardest_nodes = None

for state in itertools.permutations(range(9)):
    moves, num_nodes, error = a_star(state, manhattan_distance)

    if error:
        continue

    if len(moves) > max_length:
        max_length = len(moves)
        hardest_state = state
        hardest_moves = moves
        hardest_nodes = num_nodes

print("Najtrudniejszy stan:", hardest_state)
print("Długość ścieżki:", max_length)
print("Liczba odwiedzonych węzłów:", hardest_nodes)
print("Ścieżka ruchów:", hardest_moves)

In [ ]:
for _ in range(1000):
    state = (0,1,2,3,4,5,6,7,8)

    moves, num_nodes, error = a_star(state, manhattan_distance)
    if error:
        print("Błąd:", error)
    else:
        print("Liczba odwiedzonych węzłów:", num_nodes)
        print("Długość rozwiązania:", len(moves))
        print("Sekwencja ruchów od obecnego stanu do rozwiązania:")
        for i, move in enumerate(moves):
            print(f"{i+1}. {move}")